In [415]:
# 라이브러리 호출
import numpy as np
import pandas as pd
from datetime import datetime

In [416]:
# type Category = {
#   id: string;
#   parentId: string;
#   childrenIds: string[];
#   doIds: string[];
#   level: Integer;
# }

In [417]:
data = {
    'Id': ["root"],
    'ParentId': [""],
    'ChildrenIds': ["['서울', '경기']"],
    'DoIds':["[]"],
    'level':["0"]
}

root_df0 = pd.DataFrame(data)


In [418]:
root_df0

,Id,ParentId,ChildrenIds,DoIds,level
0,root,,"['서울', '경기']",[],0


In [419]:
# 데이터 불러오기
rawData = pd.read_csv('TPSS_region_OD3.csv', encoding='cp949', sep=",")
rawData.head()

,Date,OA,OB,DA,DB,population
0,20230920,성동구,성수2가1동,은평구,대조동,3
1,20230920,성동구,성수2가1동,은평구,불광1동,2
2,20230920,성동구,성수2가1동,은평구,불광2동,1
3,20230920,성동구,성수2가1동,은평구,수색동,2
4,20230920,성동구,성수2가1동,은평구,신사1동,3


In [420]:
filterdDf= rawData.loc[:,['OA','OB','DA','DB']]


In [421]:

# A에서 B로 이동하는 데이터 복사
output1 = filterdDf.copy()

# B에서 A로 이동하는 데이터 복사
output2 = filterdDf.copy()

# 열 이름 바꾸기
output2.columns = ['DA', 'DB','OA','OB' ]


# 두 데이터프레임 병합
merged_df = pd.concat([output1, output2], axis=0)


# (Location_1=지역1 & Location_2=지역3)와 (Location_1=지역3 & Location_2=지역1)을 같은 것으로 처리
merged_df['Unique_DB'] = merged_df[['DB', 'OB']].min(axis=1)
merged_df['Unique_OB'] = merged_df[['DB', 'OB']].max(axis=1)

merged_df['Unique_DA'] = merged_df[['DA', 'OA']].min(axis=1)
merged_df['Unique_OA'] = merged_df[['DA', 'OA']].max(axis=1)

# 중복된 데이터 제거
merged_df = merged_df.drop_duplicates(subset=['Unique_DA','Unique_DB'])

# 불필요한 열 제거
merged_df = merged_df.drop(columns=['Unique_DA','Unique_DB', 'Unique_OA','Unique_OB','DA','DB'])

#중복제거
merged_df = merged_df.drop_duplicates(subset=['OA','OB'])




In [422]:
merged_df.head(300)

,OA,OB
0,성동구,성수2가1동
56,성동구,성수2가3동
144,성동구,왕십리도선동
280,성북구,길음1동
433,성북구,삼선동
...,...,...
88710,중랑구,상봉1동
88920,평택시,세교동
89092,화성시,진안동
89893,성동구,금호4가동


In [ ]:
merged_df['OA'].unique()

In [423]:
#시 레벨 지역 리스트
regionList_df = pd.DataFrame(merged_df['OA'].unique())
# 첫번째 칼럼의 이름 조회
regionList_df.columns[0]
#  0 
# 칼럼 이름을 0에서 Region으로 변경
regionList_df=regionList_df.rename(columns={0: 'Region'})

In [425]:
# ~~시 라고 이름이 되어 있지 않은 것들은 서울 동네 리스트로,

seoul_df = regionList_df[~regionList_df['Region'].str.contains('시', case=False, regex=True)]


# ~~ 시 라고 이름이 되어 있는 것들은 경기 동네 리스트로
gyeongi_df = regionList_df[regionList_df['Region'].str.contains('시', case=False, regex=True)]

# 서울시외와 시흥시는 빼자.
gyeongi_df = gyeongi_df[gyeongi_df['Region'] != '서울시외']
gyeongi_df = gyeongi_df[gyeongi_df['Region'] != '시흥시']



In [426]:
# 경기 2는 2티어 경기도들.구 레벨의 동네를 모으기 위해 만든다.

gyeongi_df2 = gyeongi_df.copy()
gyeongi_df2[['ParentId', 'Id']] = gyeongi_df2['Region'].str.split('시', n=1, expand=True)


In [429]:
gyeongi_df1 = gyeongi_df2.copy()
# gyeongi_df1 = gyeongi_df1.drop(columns="Region")
gyeongi_df1=gyeongi_df1.rename(columns={"Id": 'ChildrenId'})
gyeongi_df1=gyeongi_df1.rename(columns={"ParentId": 'Id'})
gyeongi_df1["ParentId"] = "경기"



In [430]:
gyeongi_df1 = gyeongi_df1[['Id', 'ParentId','ChildrenId', 'Region']]

# 경기도 구 기준 테이블에 OA OB 를 합쳐 동 불러오기
gyeongi_df1 = pd.merge(gyeongi_df1, merged_df, left_on = 'Region', right_on = 'OA', how='left')

# 중복된 데이터 제거
gyeongi_df1 = gyeongi_df1.drop_duplicates(subset=['Id', 'ParentId', 'ChildrenId', 'Region', 'OA', 'OB'])
# merged_df = merged_df.drop_duplicates(subset=['Unique_DA','Unique_DB'])

# GroupBy로 OB를 묶기
gyeongi_df1 = gyeongi_df1.groupby(['Id','ParentId','ChildrenId','Region'])['OB'].apply(list).reset_index()


In [381]:
# ChildrenId의 값들..
gyeongi_df1['ChildrenId'].tolist()

['덕양구',
 '일산동구',
 '일산서구',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '분당구',
 '수정구',
 '중원구',
 '권선구',
 '영통구',
 '장안구',
 '팔달구',
 '단원구',
 '상록구',
 '동안구',
 '만안구',
 '',
 '',
 '',
 '기흥구',
 '수지구',
 '처인구',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [ ]:
# 'childrenId' 값이 있는 행과 값이 없는 행을 분리
# gyeongi_df1_with_children = gyeongi_df1[~gyeongi_df1['ChildrenId'].isna()]  # 'childrenId' 값이 있는 행
# gyeongi_df1_without_children = gyeongi_df1[gyeongi_df1['ChildrenId'].isna()]  # 'childrenId' 값이 없는 행


In [431]:
#  ChildrenId가 없는 녀석들만 따로 모으자.
# gyeongi_df1[~gyeongi_df1['ChildrenId'].isna()]
gyeongi_df1_1 = gyeongi_df1[gyeongi_df1['ChildrenId']=='']


#  ChildrenId가 있는 녀석들도 따로 모으자.
gyeongi_df1_2 = gyeongi_df1[gyeongi_df1['ChildrenId']!='']

In [432]:
gyeongi_df1_1

,Id,ParentId,ChildrenId,Region,OB
3,과천,경기,,과천시,"[과천동, 별양동, 부림동, 중앙동, 갈현동]"
4,광명,경기,,광명시,"[광명5동, 철산3동, 광명6동, 광명7동, 하안1동, 광명2동, 소하1동, 광명3..."
5,광주,경기,,광주시,"[광남동, 초월읍, 경안동, 곤지암읍]"
6,구리,경기,,구리시,"[수택1동, 수택2동, 수택3동, 인창동, 갈매동, 교문1동, 교문2동]"
7,군포,경기,,군포시,"[궁내동, 재궁동, 금정동, 광정동, 군포1동, 대야동, 산본1동, 군포2동, 산본..."
8,김포,경기,,김포시,"[김포본동, 사우동, 양촌읍, 운양동, 장기동, 구래동]"
9,남양주,경기,,남양주시,"[별내동, 별내면, 양정동, 와부읍, 조안면, 진건읍, 진접읍, 퇴계원면, 평내동,..."
10,동두천,경기,,동두천시,"[소요동, 송내동, 보산동, 생연2동]"
11,부천,경기,,부천시,"[소사본동, 소사본3동]"
23,양주,경기,,양주시,"[회천1동, 회천2동, 양주1동, 장흥면]"


In [433]:
# 여기의 Children은 없다.
gyeongi_df1_1["ChildrenIds"]="[]"

# level은 2 이다.
gyeongi_df1_1["level"]=2

# OB를 DoIds로 이름을 바꾼다.
gyeongi_df1_1=gyeongi_df1_1.rename(columns={"OB": 'DoIds'})


C:\Users\sangyoon_kim\AppData\Local\Temp\ipykernel_4632\1867079548.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gyeongi_df1_1["ChildrenIds"]="[]"
C:\Users\sangyoon_kim\AppData\Local\Temp\ipykernel_4632\1867079548.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gyeongi_df1_1["level"]=2


In [434]:
gyeongi_df1_1= gyeongi_df1_1[['Id', 'ParentId','ChildrenIds', 'DoIds', 'level']]

In [435]:
# gyeongi_df1_1 최종본
gyeongi_df1_1

,Id,ParentId,ChildrenIds,DoIds,level
3,과천,경기,[],"[과천동, 별양동, 부림동, 중앙동, 갈현동]",2
4,광명,경기,[],"[광명5동, 철산3동, 광명6동, 광명7동, 하안1동, 광명2동, 소하1동, 광명3...",2
5,광주,경기,[],"[광남동, 초월읍, 경안동, 곤지암읍]",2
6,구리,경기,[],"[수택1동, 수택2동, 수택3동, 인창동, 갈매동, 교문1동, 교문2동]",2
7,군포,경기,[],"[궁내동, 재궁동, 금정동, 광정동, 군포1동, 대야동, 산본1동, 군포2동, 산본...",2
8,김포,경기,[],"[김포본동, 사우동, 양촌읍, 운양동, 장기동, 구래동]",2
9,남양주,경기,[],"[별내동, 별내면, 양정동, 와부읍, 조안면, 진건읍, 진접읍, 퇴계원면, 평내동,...",2
10,동두천,경기,[],"[소요동, 송내동, 보산동, 생연2동]",2
11,부천,경기,[],"[소사본동, 소사본3동]",2
23,양주,경기,[],"[회천1동, 회천2동, 양주1동, 장흥면]",2


In [ ]:
# type Category = {
#   id: string;
#   name: string;
#   parentId: string;
#   childrenIds: string[];
#   doIds: string[];
#   level: Integer;
# }

In [436]:
# gyeongi_df1_2는 children에 구 가 있는 경우
# 여기서 gyeongi_df2 를 새로 나누어 구 레벨의 새 테이블 만든다.

gyeongi_df3 = gyeongi_df1_2.copy()

In [437]:
# gyeongi_df1_2 만지기.
# gyeongi_df1_2를 Id 와 ParentId 기준으로 ChildrenId 를 groupBy
gyeongi_df1_2 = gyeongi_df1_2.groupby(['Id','ParentId'])['ChildrenId'].apply(list).reset_index()

# ChildrenId를 ChildrenIds 로 이름 변경
gyeongi_df1_2=gyeongi_df1_2.rename(columns={"ChildrenId": 'ChildrenIds'})

# DoIds 에는 아무것도 없다.
gyeongi_df1_2["DoIds"]="[]"

# level은 2 이다.
gyeongi_df1_2["level"]=2


In [438]:
# gyeongi_df1_2 최종본
gyeongi_df1_2

,Id,ParentId,ChildrenIds,DoIds,level
0,고양,경기,"[덕양구, 일산동구, 일산서구]",[],2
1,성남,경기,"[분당구, 수정구, 중원구]",[],2
2,수원,경기,"[권선구, 영통구, 장안구, 팔달구]",[],2
3,안산,경기,"[단원구, 상록구]",[],2
4,안양,경기,"[동안구, 만안구]",[],2
5,용인,경기,"[기흥구, 수지구, 처인구]",[],2


In [439]:
gyeongi_df3

,Id,ParentId,ChildrenId,Region,OB
0,고양,경기,덕양구,고양시덕양구,"[대덕동, 행신3동, 행주동, 성사1동, 화전동, 화정1동, 신도동, 화정2동, 창..."
1,고양,경기,일산동구,고양시일산동구,"[마두1동, 풍산동, 마두2동, 백석1동, 백석2동, 식사동, 장항1동, 장항2동,..."
2,고양,경기,일산서구,고양시일산서구,"[대화동, 송산동, 일산1동, 일산2동, 일산3동, 주엽1동, 주엽2동, 탄현동]"
12,성남,경기,분당구,성남시분당구,"[금곡동, 백현동, 이매1동, 정자2동, 정자3동, 분당동, 서현1동, 서현2동, ..."
13,성남,경기,수정구,성남시수정구,"[수진2동, 시흥동, 신촌동, 신흥1동, 신흥2동, 고등동, 단대동, 신흥3동, 양..."
14,성남,경기,중원구,성남시중원구,"[상대원1동, 상대원2동, 상대원3동, 성남동, 은행2동, 중앙동, 하대원동, 금광..."
15,수원,경기,권선구,수원시권선구,[평동]
16,수원,경기,영통구,수원시영통구,[매탄3동]
17,수원,경기,장안구,수원시장안구,[율천동]
18,수원,경기,팔달구,수원시팔달구,"[매교동, 매산동, 인계동, 화서2동]"


In [440]:
# 불필요한 열 제거
gyeongi_df3 = gyeongi_df3.drop(columns=['ParentId','Region'])

#중복제거
gyeongi_df3 = gyeongi_df3.drop_duplicates(subset=['Id','ChildrenId'])


In [441]:

# 이름 3레벨에 맞게 전환
gyeongi_df3=gyeongi_df3.rename(columns={"Id": 'ParentId'})

gyeongi_df3=gyeongi_df3.rename(columns={"ChildrenId": 'Id'})

gyeongi_df3=gyeongi_df3.rename(columns={"OB": 'DoIds'})

# 여기의 Children은 없다.
gyeongi_df3["ChildrenIds"]="[]"

# level은 2 이다.
gyeongi_df3["level"]=3

# 순서 최종정리
gyeongi_df3= gyeongi_df3[['Id', 'ParentId','ChildrenIds', 'DoIds', 'level']]

In [442]:
# gyeongi_df3 최종본
gyeongi_df3

,Id,ParentId,ChildrenIds,DoIds,level
0,덕양구,고양,[],"[대덕동, 행신3동, 행주동, 성사1동, 화전동, 화정1동, 신도동, 화정2동, 창...",3
1,일산동구,고양,[],"[마두1동, 풍산동, 마두2동, 백석1동, 백석2동, 식사동, 장항1동, 장항2동,...",3
2,일산서구,고양,[],"[대화동, 송산동, 일산1동, 일산2동, 일산3동, 주엽1동, 주엽2동, 탄현동]",3
12,분당구,성남,[],"[금곡동, 백현동, 이매1동, 정자2동, 정자3동, 분당동, 서현1동, 서현2동, ...",3
13,수정구,성남,[],"[수진2동, 시흥동, 신촌동, 신흥1동, 신흥2동, 고등동, 단대동, 신흥3동, 양...",3
14,중원구,성남,[],"[상대원1동, 상대원2동, 상대원3동, 성남동, 은행2동, 중앙동, 하대원동, 금광...",3
15,권선구,수원,[],[평동],3
16,영통구,수원,[],[매탄3동],3
17,장안구,수원,[],[율천동],3
18,팔달구,수원,[],"[매교동, 매산동, 인계동, 화서2동]",3


In [ ]:
# 여기까지 경기도 끝

In [ ]:
# 여기서부터 서울 시작

In [443]:
seoul_df

,Region
0,성동구
1,성북구
2,송파구
9,양천구
10,양평군
11,연수구
12,영등포구
13,용산구
17,은평구
19,종로구


In [444]:
# 서울 구 기준 테이블에 OA OB 를 합쳐 동 불러오기
seoul_df2 = pd.merge(seoul_df, merged_df, left_on = 'Region', right_on = 'OA', how='left')


In [445]:
seoul_df2 = seoul_df2.groupby(['Region'])['OB'].apply(list).reset_index()


In [446]:
# 칼럼 추가하기

# 이름 2레벨에 맞게 전환
seoul_df2=seoul_df2.rename(columns={"Region": 'Id'})

seoul_df2=seoul_df2.rename(columns={"OB": 'DoIds'})

# 여기의 parentId는 서울이다.
seoul_df2["ParentId"]="서울"


# 여기의 Children은 없다.
seoul_df2["ChildrenIds"]="[]"

# level은 2 이다.
seoul_df2["level"]=2

# 순서 최종정리
seoul_df2= seoul_df2[['Id', 'ParentId','ChildrenIds', 'DoIds', 'level']]

In [447]:
seoul_df2

,Id,ParentId,ChildrenIds,DoIds,level
0,가평군,서울,[],"[가평읍, 청평면]",2
1,강남구,서울,[],"[논현1동, 삼성2동, 역삼1동, 일원본동, 논현2동, 도곡1동, 청담동, 수서동,...",2
2,강동구,서울,[],"[둔촌1동, 성내1동, 암사3동, 천호1동, 둔촌2동, 성내2동, 천호2동, 천호3...",2
3,강북구,서울,[],"[번2동, 수유1동, 번3동, 수유2동, 삼각산동, 수유3동, 삼양동, 미아동, 우...",2
4,강서구,서울,[],"[가양2동, 발산1동, 염창동, 화곡6동, 가양3동, 우장산동, 공항동, 화곡8동,...",2
5,계양구,서울,[],"[계산1동, 계산2동, 작전2동]",2
6,관악구,서울,[],"[난향동, 삼성동, 서림동, 서원동, 성현동, 신사동, 은천동, 조원동, 중앙동, ...",2
7,광진구,서울,[],"[구의2동, 자양1동, 중곡3동, 광장동, 구의3동, 자양2동, 중곡4동, 자양3동...",2
8,구로구,서울,[],"[개봉2동, 구로3동, 구로5동, 수궁동, 오류1동, 신도림동, 개봉3동, 오류2동...",2
9,금천구,서울,[],"[독산2동, 시흥1동, 시흥2동, 가산동, 독산3동, 시흥3동, 시흥4동, 독산4동...",2


In [450]:
# level1을 위한 DataFrame 만들기
root1_1= gyeongi_df1[['Id', 'ParentId']]
root1_2 = seoul_df2[['Id', 'ParentId']]

# 위아래로 합치기
root1 = pd.concat([root1_1, root1_2], axis=0, ignore_index=True)

#  중복 제거
root1 = root1.drop_duplicates(subset=['Id','ParentId'])

#  칼럼 이름 전환
root1=root1.rename(columns={"Id": 'ChildrenIds'})
root1=root1.rename(columns={"ParentId": 'Id'})

# groupBy로 합치기
root1=root1.groupby('Id')['ChildrenIds'].apply(list).reset_index()

# 칼럼 추가하기

# 여기의 parentId는 root다.
root1["ParentId"]="root"

# 여기의 DoIds는 없다.
root1["DoIds"]="[]"

# level은 2 이다.
root1["level"]=1

# 순서 최종정리
root1= root1[['Id', 'ParentId','ChildrenIds', 'DoIds', 'level']]


In [451]:
root1

,Id,ParentId,ChildrenIds,DoIds,level
0,경기,root,"[고양, 과천, 광명, 광주, 구리, 군포, 김포, 남양주, 동두천, 부천, 성남,...",[],1
1,서울,root,"[가평군, 강남구, 강동구, 강북구, 강서구, 계양구, 관악구, 광진구, 구로구, ...",[],1


In [452]:
merged_result = pd.concat([root_df0, root1], axis=0, ignore_index=True)


In [453]:
merged_result = pd.concat([merged_result, seoul_df2], axis=0, ignore_index=True)


In [454]:
merged_result = pd.concat([merged_result, gyeongi_df1_1], axis=0, ignore_index=True)


In [455]:
merged_result = pd.concat([merged_result, gyeongi_df1_2], axis=0, ignore_index=True)


In [456]:
merged_result = pd.concat([merged_result, gyeongi_df3], axis=0, ignore_index=True)


In [457]:
merged_result

,Id,ParentId,ChildrenIds,DoIds,level
0,root,,"['서울', '경기']",[],0
1,경기,root,"[고양, 과천, 광명, 광주, 구리, 군포, 김포, 남양주, 동두천, 부천, 성남,...",[],1
2,서울,root,"[가평군, 강남구, 강동구, 강북구, 강서구, 계양구, 관악구, 광진구, 구로구, ...",[],1
3,가평군,서울,[],"[가평읍, 청평면]",2
4,강남구,서울,[],"[논현1동, 삼성2동, 역삼1동, 일원본동, 논현2동, 도곡1동, 청담동, 수서동,...",2
...,...,...,...,...,...
75,동안구,안양,[],"[갈산동, 관양1동, 관양2동, 귀인동, 달안동, 범계동, 부림동, 부흥동, 비산1...",3
76,만안구,안양,[],"[석수1동, 석수2동, 안양1동, 안양2동, 안양3동, 안양4동, 안양5동, 안양6...",3
77,기흥구,용인,[],"[상하동, 신갈동, 구갈동, 동백동, 마북동, 보정동, 상갈동]",3
78,수지구,용인,[],"[풍덕천2동, 동천동, 상현1동, 성복동, 죽전2동]",3


In [459]:
merged_result.to_csv('ClusteringnAnalysis.csv', index=False, encoding='utf-8')


In [ ]:
<<======================================================>>

In [84]:
# gyeongi_df[['First_Part', 'Second_Part']] = gyeongi_df['Region'].str.split('시', 1)
gyeongi_df[['First_Part', 'Second_Part']] = gyeongi_df['Region'].str.split('시',1)


TypeError: StringMethods.split() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
gyeongi_df

In [116]:
gyeongi_df1 = gyeongi_df.copy()
gyeongi_df2 = gyeongi_df.copy()


In [ ]:
gyeongi_df1

In [178]:
# 경기도 구 기준 테이블에 OA OB 를 합쳐 동 불러오기
gyeongi_df2 = pd.merge(gyeongi_df2, merged_df, left_on = 'Region', right_on = 'OA', how='left')

# 필요없는 OA 제거하기
gyeongi_df2 = gyeongi_df2[['Id', 'ParentId', 'Region', 'OB']]

# 
gyeongi_df2


,Id,ParentId,Region,OB
0,영통구,수원,수원시영통구,매탄3동
1,상록구,안산,안산시상록구,일동
2,상록구,안산,안산시상록구,반월동
3,상록구,안산,안산시상록구,본오2동
4,상록구,안산,안산시상록구,이동
...,...,...,...,...
26481,,부천,부천시,소사본3동
26482,,부천,부천시,소사본동
26483,,부천,부천시,소사본3동
26484,,부천,부천시,소사본동


In [185]:
gyeongi_df2.groupby(['Id','ParentId','Region'])['OB'].apply(list).reset_index()



,Id,ParentId,Region,OB
0,,과천,과천시,"[과천동, 별양동, 부림동, 중앙동, 갈현동, 과천동, 별양동, 부림동, 중앙동, ..."
1,,광명,광명시,"[광명5동, 철산3동, 광명6동, 광명7동, 하안1동, 광명2동, 소하1동, 광명3..."
2,,광주,광주시,"[광남동, 초월읍, 경안동, 곤지암읍, 광남동, 초월읍, 경안동, 곤지암읍, 광남동..."
3,,구리,구리시,"[수택1동, 수택2동, 수택3동, 인창동, 갈매동, 교문1동, 교문2동, 수택1동,..."
4,,군포,군포시,"[궁내동, 재궁동, 금정동, 광정동, 군포1동, 대야동, 산본1동, 군포2동, 산본..."
5,,김포,김포시,"[김포본동, 사우동, 양촌읍, 운양동, 장기동, 구래동, 김포본동, 사우동, 양촌읍..."
6,,남양주,남양주시,"[별내동, 별내면, 양정동, 와부읍, 조안면, 진건읍, 진접읍, 퇴계원면, 평내동,..."
7,,동두천,동두천시,"[소요동, 송내동, 보산동, 생연2동, 소요동, 송내동, 보산동, 생연2동, 소요동..."
8,,부천,부천시,"[소사본동, 소사본3동, 소사본동, 소사본3동, 소사본동, 소사본3동, 소사본동, ..."
9,,양주,양주시,"[회천1동, 회천2동, 양주1동, 장흥면, 회천1동, 회천2동, 양주1동, 장흥면,..."


In [ ]:
gyeongi_df1 = gyeongi_df1[['Id', 'ParentId','ChildrenId']]


gyeongi_df2.groupby('Id')['childrenId'].apply(list).reset_index()


In [113]:
import pandas as pd

# 예시 데이터프레임 생성
data = {'Text': ['Apple', 'Banana', 'Cherry', 'Date', 'Fig', 'Grape']}
df = pd.DataFrame(data)

# 'a'를 기준으로 텍스트를 나누어 두 개의 열 생성
df[['First_Part', 'Second_Part']] = df['Text'].str.split('(a)', n=1, expand=True)

# 결과 출력
print(df)

ValueError: Columns must be same length as key

In [109]:
import pandas as pd

# 예시 데이터프레임 생성
data = {'Text': ['Apple', 'Banana', 'Cherry', 'Date', 'Fig', 'Grape']}
df = pd.DataFrame(data)

# 'a'를 기준으로 텍스트를 나누어 두 개의 열 생성
df[['First_Part', 'Second_Part']] = df['Text'].str.split('a', 1)

# 결과 출력
print(df)

TypeError: StringMethods.split() takes from 1 to 2 positional arguments but 3 were given

In [153]:
import pandas as pd
from io import StringIO

data = """Id,parentId,childrenId
안산,서울,상록구
안양,서울,동안구
안양,서울,만안구
양주,서울,
용인,서울,기흥구
용인,서울,수지구
용인,서울,처인구
의정부,서울,
파주,서울,
성남,서울,분당구
성남,서울,수정구
성남,서울,중원구
수원,서울,장안구
여주,서울,
하남,서울,
수원,서울,팔달구"""

# CSV 데이터를 데이터프레임으로 읽기
df = pd.read_csv(StringIO(data), encoding='utf-8')

# 'Id'에 따른 'childrenId'를 리스트로 묶기
result = df.groupby('Id')['childrenId'].apply(list).reset_index()

# 결과 출력
print(result)

    Id       childrenId
0   성남  [분당구, 수정구, 중원구]
1   수원       [장안구, 팔달구]
2   안산            [상록구]
3   안양       [동안구, 만안구]
4   양주            [nan]
5   여주            [nan]
6   용인  [기흥구, 수지구, 처인구]
7  의정부            [nan]
8   파주            [nan]
9   하남            [nan]


In [154]:
# 'doId' 열 생성
df['doId'] = df['childrenId'].apply(lambda x: ["가동", "나동", "다동"] if pd.isna(x) else [])


In [161]:


pd.merge(df, merged_df, left_on = 'childrenId', right_on = 'OA', how='left')


,Id,parentId,childrenId,OA,OB
0,안산,서울,상록구,NaN,NaN
1,안양,서울,동안구,NaN,NaN
2,안양,서울,만안구,NaN,NaN
3,양주,서울,NaN,NaN,NaN
4,용인,서울,기흥구,NaN,NaN
5,용인,서울,수지구,NaN,NaN
6,용인,서울,처인구,NaN,NaN
7,의정부,서울,NaN,NaN,NaN
8,파주,서울,NaN,NaN,NaN
9,성남,서울,분당구,NaN,NaN
